### create a model to run through radlite

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from radmc3dPy import *
%matplotlib inline

In [13]:
# model parameters
mstar = ['1.0*ms']        # stellar mass
tstar = [4000.0]          # Teff (K)
rstar = ['2.0*rs']        # stellar radius

mdisk = '1e-4*ms'         # dust disk mass
dusttogas = 0.01          # dust to gas ratio
rin = '0.05*au'           # inner disk radius
rdisk = '100*au'          # outer disk radius
gap_rin='[0.1*au]'        # gap inner radius
gap_rout='[40*au]'        # gap outer radius

In [14]:
# set up the model and write out new style files for radmc
# note you need to have dustkappa_silicate.inp in this directory
analyze.writeDefaultParfile('ppdisk')
model_new = setup.problemSetupDust(model='ppdisk', mstar=mstar, rstar=rstar, tstar=tstar,
                                   mdisk=mdisk, dusttogas=dusttogas,
                                   rin=rin, rdisk=rdisk,
                                   gap_rin=gap_rin, gap_rout=gap_rout, gap_drfact='[1e-5]',
                                   nz='0', binary=False)

Writing problem_params.inp
Writing problem_params.inp
Writing dustopac.inp
Writing wavelength_micron.inp
Writing amr_grid.inp
Writing stars.inp
-------------------------------------------------------------
Luminosities of radiation sources in the model :
Reading wavelength_micron.inp
As calculated from the input files :
Stars : 
  Star #0 + hotspot        : 3.564346e+33
Continuous starlike source : 0.000000e+00
 
-------------------------------------------------------------
Writing dust_density.inp
Writing radmc3d.inp


In [4]:
os.system("radmc3d mctherm")

  
      WELCOME TO RADMC-3D: A 3-D CONTINUUM AND LINE RT SOLVER    
                                                                 
                          VERSION 2.0                            
                                                                 
                (c) 2008-2020 Cornelis Dullemond                 
                                                                 
       Please feel free to ask questions. Also please report     
        bugs and/or suspicious behavior without hestitation.     
      The reliability of this code depends on your vigilance!    
                    dullemond@uni-heidelberg.de                  
                                                                 
   To keep up-to-date with bug-alarms and bugfixes, register to  
                     the RADMC-3D forum:                         
            http://radmc3d.ita.uni-heidelberg.de/phpbb/          
                                                                 
       

0

In [5]:
# make a directory for the radmc outputs for radlite to use
outputdir = "./radmc_outputs"
print(f"Moving radmc files to output directory {outputdir}")
if os.path.exists(outputdir):
    print("Will overwrite existing files")
else:
    print("Directory does not exist; will create")
    os.makedirs(outputdir)

Moving radmc files to output directory ./radmc_outputs
Directory does not exist; will create


In [6]:
# translate temperature array to old radmc format
data = analyze.readData(ddens=True, dtemp=True)
Tdust = data.dusttemp[:,:,0,0]
nr, nt = Tdust.shape
fname = "dusttemp_final.dat"
with open(fname, "w") as wfile:
    print("Writing " + fname)
    wfile.write(f"   1    {nr:d}   {nt//2:d}   1\n")
    wfile.write(" \n")
    wfile.write("   1\n")
    for i in range(nr):
        for j in range(nt//2):
            wfile.write(f"{Tdust[i,j]:.7f}\n")
os.system("mv "+fname+" radmc_outputs/")
            
fname = "dusttemp.info"
with open(fname, "w") as wfile:
    print("Writing " + fname)
    wfile.write("  -2\n")
    wfile.write("   1\n")
    wfile.write("   1\n")
    wfile.write("   1\n")
    wfile.write("   1\n")
os.system("mv "+fname+" radmc_outputs/")

Reading amr_grid.inp
Reading wavelength_micron.inp
Reading dust_density.inp
Reading dust_temperature.dat
Writing dusttemp_final.dat
Writing dusttemp.info


0

In [15]:
# write out the old format radmc files since radlite needs these
# note that this overwrites the new format dustopac.inp so first save that and rename it back afterwards!
os.system("mv dustopac.inp dustopac_newformat.inp")

model_old = setup.problemSetupDust(model='ppdisk', mstar=mstar, rstar=rstar, tstar=tstar,
                                   mdisk=mdisk, dusttogas=dusttogas,
                                   rin=rin, rdisk=rdisk,
                                   gap_rin=gap_rin, gap_rout=gap_rout, gap_drfact='[1e-5]',
                                   nz='0', binary=False, old=True)

filelist = ["problem_params.inp", "dustdens.inp", "dustopac.inp", "frequency.inp", "dustopac_1.inp", "radius.inp", "theta.inp", "starinfo.inp", "starspectrum.inp"]
for file in filelist:
    os.system("mv "+file+" radmc_outputs/")
    
os.system("mv dustopac_newformat.inp dustopac.inp")

Writing problem_params.inp
Writing dustopac.inp
Writing frequency.inp
Reading dustkappa_silicate.inp
Writing dustopac_1.inp
Writing frequency.inp
Writing radius.inp
Writing theta.inp
Writing starinfo.inp
Writing starspectrum.inp


0

In [ ]:
filelist = ["run_radlite.ipynb", "model.json", "spectrum.json", "line.inp", "molecule_co.inp"]
for file in filelist:
    os.system("mv "+file+" radmc_outputs/")